# ДЗ2
### Екатерина Кострыкина




Возьмем данные про музыкальные инструменты

In [3]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Musical_Instruments.json.gz

--2021-12-06 20:09:11--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2460495 (2.3M) [application/x-gzip]
Saving to: ‘reviews_Musical_Instruments_5.json.gz.1’

reviews_Musical_Ins 100%[===================>]   2.35M  3.29MB/s    in 0.7s    

2021-12-06 20:09:12 (3.29 MB/s) - ‘reviews_Musical_Instruments_5.json.gz.1’ saved [2460495/2460495]

--2021-12-06 20:09:12--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Musical_Instruments.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172497344 (165M) [application/octet-stream]
Saving to: ‘meta_Musical_Instruments.json.g

In [4]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        true = True
        false = False
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/content/reviews_Musical_Instruments_5.json.gz')
meta = getDF('/content/meta_Musical_Instruments.json.gz')

In [5]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"
...,...,...,...,...,...,...,...,...,...
10256,A14B2YH83ZXMPP,B00JBIVXGC,Lonnie M. Adams,"[0, 0]","Great, just as expected. Thank to all.",5.0,Five Stars,1405814400,"07 20, 2014"
10257,A1RPTVW5VEOSI,B00JBIVXGC,Michael J. Edelman,"[0, 0]",I've been thinking about trying the Nanoweb st...,5.0,"Long life, and for some players, a good econom...",1404259200,"07 2, 2014"
10258,AWCJ12KBO5VII,B00JBIVXGC,Michael L. Knapp,"[0, 0]",I have tried coated strings in the past ( incl...,4.0,Good for coated.,1405987200,"07 22, 2014"
10259,A2Z7S8B5U4PAKJ,B00JBIVXGC,"Rick Langdon ""Scriptor""","[0, 0]","Well, MADE by Elixir and DEVELOPED with Taylor...",4.0,Taylor Made,1404172800,"07 1, 2014"


In [6]:
meta

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Musical Instruments, Drums & Percussion, Hand...",,[Cricket Rubbing the spine with the wooden sti...,,Wooden Percussion 2 Piece Set of 3 Inch Cricke...,"[B00NP8GYVS, B00NP80XMO, B00NP8M098]",,WADSUWAN SHOP,"[Wood percussion, Owl whistle*, Includes woode...","[>#141,729 in Musical Instruments (See Top 100...",[],Musical Instruments,,"December 2, 2013",,0000989983,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Musical Instruments, Drums & Percussion, Hand...",,[Frog - Rubbing its spine with the wooden stic...,,"Wooden Percussion 3 Piece Set Frog, Cricket an...","[B00NP8GYVS, B00NP80XMO, B01MY48HK5, B00AZZ1AJ...",,WADSUWAN SHOP,"[Wood percussion, Small 3 inches, Creates orig...","[>#1,622 in Musical Instruments (See Top 100 i...",[],Musical Instruments,,"December 2, 2013",$0.91,0000098906,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Musical Instruments, Instrument Accessories, ...",,[Vivaldi's famous set of four violin concertos...,,Hal Leonard Vivaldi Four Seasons for Piano (Or...,[],,Hal Leonard,"[., ., .]","[>#330,653 in Musical Instruments (See Top 100...",[],Musical Instruments,,"May 10, 2011",$62.93,0041291905,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
3,[],,"[The Turn of the Screw (op. 54) vocal score, p...",,The Turn of the Screw (vocal score),"[0486266842, 0793507669, 0393008789, 142341280...",,Boosey &amp; Hawkes,[],"[>#86,354 in Musical Instruments (See Top 100 ...",[],Musical Instruments,,"May 23, 2007",$107.79,0060015500,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,[],,[],,Suite for Organ (including the Trumpet Volunta...,[],,,[],"[>#482,025 in Musical Instruments (See Top 100...",[],Musical Instruments,,"February 8, 2013",,0193757710,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120305,"[Musical Instruments, Amplifiers & Effects, Gu...",,[This packages includes:<br> <br> 10 pcs 3pdt ...,,10 pcs 3pdt Stomp Footswitch incl PCB incl met...,[],,3pdt,[5 pcs 9 Pole Footswitch for True Bypass Switc...,"[>#23,334 in Musical Instruments (See Top 100 ...",[],Musical Instruments,,"June 25, 2016",$7.99,B01HJDOF2Y,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
120306,"[Musical Instruments, Amplifiers & Effects, Gu...",,[This packages includes:<br> <br> 5 pcs 3pdt T...,,"5 pcs 3pdt Stomp Footswitch incl. PCB, metal w...","[B015334J9G, B01L6GU858, B003HLW14W, B012CF181...",,3pdt,[5 pcs 9 Pole Footswitch for True Bypass Switc...,"[>#40,026 in Musical Instruments (See Top 100 ...",[],Musical Instruments,,"June 25, 2016",$0.91,B01HJDJ1PA,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
120307,"[Musical Instruments, Drums & Percussion, Hand...",,[These high quality clear crystal singing bowl...,,Optical Clear Quartz Crystal Singing Bowl Note...,[],,Crystal Energy Bowls,[High quality clear crystal singing bowl is wi...,"[>#481,984 in Musical Instruments (See Top 100...",[],Musical Instruments,,"June 25, 2016",,B01HJEFFTK,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
120308,"[Musical Instruments, Drums & Percussion, Hand...",,[Frog - Rubbing its spine with the wooden stic...,,2x Deluxe Small 2&quot; Wood Frog Guiro Rasp -...,[],,RA,"[Wood percussion, Small 2 inches, Packs of 2 P...","[>#76,917 in Musical Instruments (See Top 100 ...","[B00NP80XMO, B00NP8GYVS, 0000098906, B0149IWDS...",Musical Instruments,"class=""a-bordered a-horizontal-strip

## (3 балла) Предложите 3 способа найти упоминания товаров в отзывах. Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара. Могут помочь заголовки и дополнительные данные с Amazon (Metadata здесь) Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?

1/ Так как у нас есть метаданные и там есть столбец title, можно найти самые частотные слова в этом столбце и самые частотные слова в столбце reviewText нашего датасета. Далее из самых частотных мы выбираем нужные слова и таким образом получаем список сущностей, с которыми можно дальше работать.
Плюсы: скорее всего в нашем датасете про музыкальные инструменты самыми частотными будут слова типа guitar violent piano, как и в других датасетах на другие темы, что будет являться правильным и мы действительно выделим нужные сущности. Минусы: возможно будет много мысорных слов, общих для всех товаров.

2/ Можно составить морфологические шаблоны, например NOUN+NOUN или ADJ+NOUN, выделить их из текстов отзывов и далее отсортировать по частотности.

3/ Также можно попробовать искать названия инструментов и их брендов, используя готовые NER решения. И искать шаблоны типа NOUN+ORG в данных. Далее отсортировать по частотности

Минусы этих подходов: может оказаться сложно фильтровать по частотности, в подходе с NER можем не выделить все марки товаров и придется добавлять вручную.

4/ Например к нас пришел продавец гитар Yamaha и хочет узнать про свой товар. Мы можем из всех отзывов оставить только те, в которых присутсвует слова "Yamaha" и дальше работать только с ними, так как нет необходимости обрабатывать отзывы о гитарах другой фирмы или вообще о скрипках. В этих отзывах можно найти n-граммы со словом "guitar" или запрашивать конкретные названия моделей. Минусы: придется вручнную запрашивать у клиента названия и вбивать их в модель, может занять много времени и сил. Плюсы: зная точную марку гитары будет намного проще найти результат по ней.

## (2 балла) Реализуйте один из предложенных вами способов.



Реализуем 1 способ

In [7]:
from collections import Counter
import nltk
nltk.download('stopwords')
from string import punctuation
from nltk.corpus import stopwords
stops = stopwords.words('english')
from nltk.tokenize import word_tokenize
import spacy
nlp = spacy.load('en_core_web_sm')
from tqdm.notebook import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
def find_words(data):
  tokens = []
  for text in tqdm(data):
    text = text.lower()
    text = nlp(text)
    for word in text:
      pos = word.pos_
      if pos == "NOUN" and word.lemma_ not in stops:
        tokens.append(word.lemma_)
  return tokens

In [9]:
noun_words = find_words(df['reviewText'])

  0%|          | 0/10261 [00:00<?, ?it/s]

In [10]:
Counter(noun_words).most_common(30)

[('guitar', 6380),
 ('string', 4460),
 ('pedal', 3559),
 ('sound', 2410),
 ('amp', 2242),
 ('price', 2206),
 ('pick', 1864),
 ('time', 1828),
 ('one', 1821),
 ('quality', 1768),
 ('tone', 1753),
 ('cable', 1651),
 ('tuner', 1483),
 ('thing', 1462),
 ('strap', 1461),
 ('product', 1315),
 ('stand', 1296),
 ('year', 1250),
 ('way', 1064),
 ('case', 1040),
 ('bit', 1010),
 ('lot', 1005),
 ('instrument', 989),
 ('problem', 940),
 ('bass', 920),
 ('fender', 840),
 ('effect', 826),
 ('end', 771),
 ('power', 716),
 ('review', 705)]

In [11]:
noun_words_title = find_words(meta['title'])

  0%|          | 0/120310 [00:00<?, ?it/s]

In [12]:
Counter(noun_words_title).most_common(30)

[('guitar', 21288),
 ('case', 7647),
 ('string', 6193),
 ('cable', 5866),
 ('pack', 4545),
 ('bag', 4333),
 ('microphone', 4176),
 ('bass', 3774),
 ('pedal', 3614),
 ('inch', 3507),
 ('size', 3020),
 ('light', 3005),
 ('effect', 2823),
 ('stand', 2777),
 ('instrument', 2769),
 ('violin', 2767),
 ('pick', 2717),
 ('set', 2674),
 ('music', 2663),
 ('strap', 2585),
 ('audio', 2538),
 ('system', 2462),
 ('gold', 2278),
 ('style', 2199),
 ('power', 2192),
 ('keyboard', 2181),
 ('leather', 2153),
 ('kit', 2148),
 ('drum', 2006),
 ('foot', 1952)]

(1 балл) Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

In [13]:
vocab_list = ['guitar', 'pedal', 'cable', 'microphone','instrument', 'set']

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

c_vec = TfidfVectorizer(max_features=50000, stop_words=stops, ngram_range=(1, 5))

ngrams = c_vec.fit_transform(df['reviewText'])

vocab = c_vec.vocabulary_

count_values = ngrams.toarray().sum(axis=0)
ngrams = []
for ng_count, ng_text in sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True):
  for word in vocab_list:
    if word in ng_text.split():
      ngrams.append(ng_text)

In [15]:
ngrams[:30]

['guitar',
 'pedal',
 'cable',
 'set',
 'instrument',
 'microphone',
 'acoustic guitar',
 'electric guitar',
 'pedal board',
 'guitar strings',
 'guitar strap',
 'guitar stand',
 'guitar case',
 'guitar picks',
 'play guitar',
 'guitar player',
 'playing guitar',
 'bass guitar',
 'one guitar',
 'new guitar',
 'guitar sound',
 'guitar bass',
 'great guitar',
 'guitar amp',
 'use guitar',
 'put guitar',
 'quality cable',
 'holds guitar',
 'classical guitar',
 'distortion pedal']

посмотрим на прилагательные рядом с существительными

In [16]:
def find_ADJ_words(data):
  tokens = []
  for pair in tqdm(data):
    for word in nlp(pair):
      if word.pos_ == "ADJ":
        tokens.append(pair)
  return tokens

In [17]:
adj_ngrams = find_ADJ_words(ngrams)

  0%|          | 0/2997 [00:00<?, ?it/s]

In [18]:
adj_ngrams[:20]

['acoustic guitar',
 'electric guitar',
 'great guitar',
 'classical guitar',
 'great pedal',
 'easy set',
 'good cable',
 'good pedal',
 'good guitar',
 'good quality cable',
 'pedal great',
 'cable good',
 'best guitar',
 'pedal good',
 'stringed instrument',
 'old guitar',
 'overdrive pedal',
 'better guitar',
 'local guitar',
 'cheap guitar']

## (3 балла) Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика ранжирует выше коллокации, подходящие для отчёта).

In [19]:
from nltk.collocations import *

In [20]:
list_ngrams = []
for i in ngrams:
  list_ngrams.append(i.split())

In [21]:
finder2 = BigramCollocationFinder.from_documents(list_ngrams)
finder3 = TrigramCollocationFinder.from_documents(list_ngrams)

In [22]:
finder2.apply_freq_filter(3)
finder3.apply_freq_filter(3)

In [23]:
measures2 = nltk.collocations.BigramAssocMeasures()
measures3 = nltk.collocations.TrigramAssocMeasures()

T-score

In [24]:
bigrams_t = finder2.score_ngrams(measures2.student_t)
trigrams_t = finder3.score_ngrams(measures3.student_t)

In [25]:
bigrams_t[:10]

[(('acoustic', 'guitar'), 5.290791546479768),
 (('pedal', 'board'), 4.795549239556227),
 (('electric', 'guitar'), 4.183899016723015),
 (('guitar', 'sound'), 3.068939164299462),
 (('guitar', 'case'), 3.0328578922891882),
 (('new', 'guitar'), 2.77900098324011),
 (('cable', '10'), 2.74937343358396),
 (('drum', 'instrument'), 2.72209527795122),
 (('quality', 'cable'), 2.636591478696742),
 (('adjustable', 'microphone'), 2.587651509031426)]

In [26]:
trigrams_t[:10]

[(('guitar', 'cable', '10'), 2.767318144288624),
 (('guitar', 'sound', 'like'), 2.6320686724480855),
 (('periodically', 'unscrewing', 'guitar'), 2.448705714198354),
 (('piece', 'typical', 'guitar'), 2.448705714198354),
 (('guitar', 'pick', 'bigger'), 2.448470505622907),
 (('typical', 'guitar', 'pick'), 2.447791014182726),
 (('studio', 'album', 'guitar'), 2.447608074179601),
 (('unscrewing', 'guitar', 'body'), 2.4475296713211185),
 (('guitar', 'body', 'one'), 2.440081399765291),
 (('gtr210', 'guitar', 'cable'), 2.4247928423612226)]

PMI

In [27]:
bigrams_pmi = finder2.score_ngrams(measures2.pmi)
trigrams_pmi = finder3.score_ngrams(measures3.pmi)

In [28]:
bigrams_pmi[:10]

[(('relying', 'external'), 10.810169937664657),
 (('iii', 'carbon'), 10.488241842777295),
 (('les', 'paul'), 10.488241842777295),
 (('microphoneor', 'dm80'), 10.488241842777295),
 (('cent', 'piece'), 10.225207436943501),
 (('voodoo', 'lab'), 10.225207436943501),
 (('carbon', 'fiber'), 10.002815015607055),
 (('alesis', 'qs8'), 10.002815015607052),
 (('cases', 'appear'), 10.002815015607052),
 (('msa7040tb', 'adjustable'), 9.810169937664657)]

In [29]:
trigrams_pmi[:10]

[(('iii', 'carbon', 'fiber'), 20.491056858384354),
 (('damaged', 'cases', 'appear'), 20.00563003121411),
 (('cent', 'piece', 'typical'), 19.713449279720802),
 (('mic', 'alesis', 'qs8'), 19.00563003121411),
 (('waves', 'classic', 'series'), 18.812984953271716),
 (('stage', 'msa7040tb', 'adjustable'), 18.7134492797208),
 (('guitarists', 'studio', 'album'), 18.450414873887006),
 (('pick', 'bigger', 'size'), 17.919900157188227),
 (('locks', 'periodically', 'unscrewing'), 17.597972062300865),
 (('relying', 'external', 'microphone'), 16.51181541855115)]

CHI_SQ

In [30]:
bigrams_chi = finder2.score_ngrams(measures2.chi_sq)
trigrams_chi = finder3.score_ngrams(measures3.chi_sq)

In [31]:
bigrams_chi[:10]

[(('relying', 'external'), 5385.749686585876),
 (('carbon', 'fiber'), 5128.570433328689),
 (('msa7040tb', 'adjustable'), 4486.873693743904),
 (('iii', 'carbon'), 4307.9994985374005),
 (('les', 'paul'), 4307.9994985374005),
 (('microphoneor', 'dm80'), 4307.9994985374005),
 (('periodically', 'unscrewing'), 4306.797993311037),
 (('piece', 'typical'), 4306.797993311037),
 (('damaged', 'cases'), 3661.4263420073084),
 (('cent', 'piece'), 3589.499373171751)]

In [32]:
trigrams_chi[:10]

[(('iii', 'carbon', 'fiber'), 4422061.914089985),
 (('damaged', 'cases', 'appear'), 3158614.6714010607),
 (('cent', 'piece', 'typical'), 2580134.548431244),
 (('mic', 'alesis', 'qs8'), 1579012.9286204143),
 (('waves', 'classic', 'series'), 1381635.1586495272),
 (('stage', 'msa7040tb', 'adjustable'), 1290246.322612379),
 (('guitarists', 'studio', 'album'), 1075053.306656024),
 (('pick', 'bigger', 'size'), 744786.0247460481),
 (('locks', 'periodically', 'unscrewing'), 596244.3565659021),
 (('relying', 'external', 'microphone'), 280344.32536605874)]

Для биграмм лучше всего справилась метрика - T-score, а для триграмм все метрики показали похожий результат. Так что T-score можем оставить как наиболее подходящую для отчета.

In [33]:
def group_collacations(ne, data):
  group = []
  for i in data:
    if ne in i[0]:
      group.append(' '.join(i[0]))
  return group

In [34]:
vocab_list

['guitar', 'pedal', 'cable', 'microphone', 'instrument', 'set']

In [35]:
all_results = sorted(bigrams_t + trigrams_t, key=lambda x: -x[1])

In [36]:
group_collacations('guitar', all_results)[:5]

['acoustic guitar',
 'electric guitar',
 'guitar sound',
 'guitar case',
 'new guitar']

In [37]:
group_collacations('microphone', all_results)[:5]

['adjustable microphone',
 'boom microphone',
 'msa7040tb adjustable microphone',
 'tripod boom microphone',
 'external microphone']

In [38]:
group_collacations('cable', all_results)[:5]

['guitar cable 10',
 'cable 10',
 'quality cable',
 'gtr210 guitar cable',
 'inside cable']

In [39]:
group_collacations('pedal', all_results)[:5]

['pedal board', 'distortion pedal', 'lab pedal', 'pedal power', 'volume pedal']

In [40]:
group_collacations('instrument', all_results)[:5]

['drum instrument',
 'series instrument',
 'drum instrument microphoneor',
 'instrument microphoneor',
 'instrument clip']

## Бонус (2 балла): если придумаете способ объединить синонимичные упоминания (например, "Samsung Galaxy Watch", "watch", "smartwatch")

Можно взять предобученную векторную модель типа word2vec, затем дообучить на наших данных. Так как упоминания типа "Samsung Galaxy Watch", "watch", "smartwatch" должны встречаться в похожих контекстах, в полученной моделе данные упоминания должны будут находиться рядом друг с другом (например по косинусной близости векторов). Таким образом, по запросу "watch" мы сможем получить и упоминания, употребляющееся вместо него в том же контексте.

Еще можно добавить сюда кластеризацию, возможно эти упоминания окажутся в одном кластере